In [ ]:
!pip install openai
!pip install pymupdf # Install PyMuPDF which includes the correct fitz module.
!pip install -U pip setuptools wheel
!pip uninstall -U spacy
!pip install -U spacy==3.7.6
import spacy
import os
import pandas as pd
nlp = spacy.load("en_core_web_sm")

In [23]:
from pypdf import PdfReader
import os
start='abstract'
end='references'

start_page=0
main_dir="/content/Docs/"
def find_index(text,target):
    try:
        return text.index(target)
    except:
        return -1

for files in os.listdir(main_dir):
    passage=""
    found=False
    path=main_dir+files
    if files[-3:]!="pdf":
        continue
    reader = PdfReader(path)
    number_of_pages = len(reader.pages)
    for page_num in range(number_of_pages):
        page = reader.pages[page_num]
        text = [texts.lower() for texts in page.extract_text().split()]
        start_idx=find_index(text,start)+1
        end_idx=find_index(text,end)
        if found:
            if start_idx == -1 and end_idx==-1:
                passage += " ".join(text)
            elif end_idx != -1 and start_idx == -1:
                passage += " ".join(text[:end_idx])
            else:
                passage += " ".join(text[start_idx:end_idx])
        elif not found:
            if start_idx != -1 and end_idx != -1:
                found=True
                passage += " ".join(text[start_idx:end_idx])
            elif start_idx == -1 and end_idx != -1:
                found=True
                passage += " ".join(text[:end_idx])
            elif start_idx == -1 and end_idx == -1:
                continue
            else:
                found=True
                passage += " ".join(text[start_idx:])
    with open(main_dir+"text/"+files[:-3]+".txt", 'w') as f:
        f.write(passage)




direct pdf w/ llm

In [39]:
API_KEY=''
modeltype= 'gpt-4-turbo-preview'
main_dir="Docs/"

In [6]:
!pwd

/Users/rishikasrinivas/KnowledgeGraphMentalHealth


In [ ]:
#pdf llm
import os
from openai import OpenAI

client= OpenAI( api_key= API_KEY)

for filename in os.listdir(main_dir):
    print(filename[-3:])
    if filename[-3:] != 'pdf':
        continue
    filename= main_dir + filename
    assistant_name= 'Clinicial Assistant'
    
    instructions=  """
      json: You are to extract important entities, their definitions, overarching ideas,  and their relationships with one another from this paper. Don't copy the sentence
      word for word and split it up. Do not take all random relationships. Identify relationships that can help clinicians with diagnosis. GET THE 
      MAIN IDEA!!. Will/May need to look across sections.
      
      Understand the sentence and determine the relationships between the subject and objects
    
      DO NOT use the word 'Depression' ALONE as a subject or object. Each block of text emphasizes different types of depression and you must include the type of depression the relationship is referring to.
      If the text is referring to all types of depression for that relationship, label it as "All Depression"
      The idea is that these relationships can be pieced together into a knowledge graph that a clinician can trace. Keep this in mind as you extract.
    
      Do not use any relationships other than these: 
        1. treatment:  IPT --> treatment for -> depression
        2. symptom: sadness -> symptom of -> depression
        3. example: medA --> example of -> antidepressant
        4. not effective: medA -> not effective for --> depression
        5. effective: medA -> effective for -> depression
        6. more effective: medA -> more effective than -> medB
        7. less effective: medA -> less effective than -> medB
        8. side effects: sneezing --> side effect of -> medA
        9. treatment:  meditate --> treatment to -> reduce depression
        10. Same efficacy: medA -> Same efficacy as -> no meds
        11. Recommended first-line treatment: TreatA --> Recommended first-line treatment for --> illnesses
        12. High Efficiency: MedA -> High Efficiency for -> DiseaseA
        13. Less commonly used: MedB -> Less commonly used for -> Depression
        14. correlated: DiseaseA -> correlated with -> ProblemA
        15. More common than ... in ... : DiseaseA -> More common than -> DiseaseB -> in -> younger patients
        16. Concurrent comorbidity: Disease A -> Concurrent comorbidity -> IllnessC
        17. Results: DiseaseA -> results -> faster treatment in young people

    Each of these relationships is the base from which you can deviate a bit. Ex you can say treatment for, treatment with, treats, etc but stick 
    to the base idea of treating. Same applies for all relations
    
      Example:  As shown in Table 2, remission rates were significantly lower in patients with anxious depression,
      according to both the HAM-D criterion (22.2% versus 33.4%) and the QIDS-SR criterion (27.5% versus 38.9%). 
      Response rates were also significantly lower for patients with anxious depression (41.7% versus 52.8%). "
    
      Gives the rel(s): Anxious Depression -> finding -> low remission rates  and Anxious Depression -> finding -> low response rates 
      
      Present 1 dataframe in an easily readable and visually appealing format.
      The dataframe should explain the relationships between all subjects.
      It should have a 4 headers, subj, rel, obj, reference. Reference should hold the exact sentence(s) (in quotes) that you used to get the relationship
      If you have object2 as a header, you must have a non-empty rel2 as a header. If thats not possible place the relationship in the next row
    
    
      return a pandas dataframe structure called paper1
      Return a form of a dictionary of lists.
      Avoid entities and relationships that are more than a few words. If you need to list multiple drug names or treatments, list them in different rows


    """
    
    assistant= client.beta.assistants.create(
    name= assistant_name,
    instructions= instructions,
    model= modeltype,
    tools= [{"type":"file_search"}])
    
    message_file= client.files.create(
    file=open(filename, "rb"),
    purpose="assistants")
    thread= client.beta.threads.create(
    messages=[
        {
            "role":"user",
            "content": instructions,
            "attachments":[
                {"file_id": message_file.id,
                "tools":[{"type":"file_search"}]}
            ],
    
        }
    ]
                 )
    
    run= client.beta.threads.runs.create_and_poll(thread_id= thread.id, assistant_id= assistant.id)
    
    messages= list(client.beta.threads.messages.list(thread_id=thread.id, run_id= run.id))
    message_content= messages[0].content[0].text
    annotations= message_content.annotations
    citations=[]
    for index, annotation in enumerate(annotations):
              message_content.valye= message_content.value .replace(annotation.text,""). replace(".", "")
              if file_citation :=getattr(annotations, "file_citation", None):
                  cited_file= client.files.retrieve(file_citation.file.id)
                  citations.append(f"[{index}] {cited_file.filename}")
    cand_name = message_content.value.replace(".", ""). strip()
    print(cand_name)

nlp processing w/ llm

In [21]:
import fitz
import os
import pandas as pd
texts=[]
# Opening the PDF file
skip=3
for doc in os.listdir(main_dir):
    print(doc[-3:])
    if doc[-3:] != 'pdf':
        continue
    doc=fitz.open(main_dir+doc)
    # Extracting text from all pages
    all_text = []
    for page_num in range(0,len(doc),skip):
        tex=""
        for p in range(page_num, page_num+skip):
          if p >= len(doc):
            break
          page = doc[p]
          text = page.get_text().replace('\n', ' ').lower()
          docs=nlp(text)
          text=" ".join([token.lemma_ for token in docs])
          tex += text
        all_text.append([page_num + 1, tex])
        df = pd.DataFrame(all_text, columns=['Page', 'Text'])
        for text in df['Text'].to_list():
            if 'References' in text or 'Received' in text:
                break
            texts.append(text)

pdf
txt
pdf
nts
txt
txt
pdf


In [27]:
!pip install torch
import torch
torch.cuda.empty_cache()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 39.4 MB/s eta 0:00:00a 0:00:01


In [38]:
import pandas as pd
client= OpenAI( api_key= '')
client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": 'prompt',
            },
        {
            "role": "user",
            "content": f'answer the {prompt}',
            }
        ],
    model= modeltype,
)

APIConnectionError: Connection error.

In [13]:
from openai import OpenAI
client= OpenAI( api_key= API_KEY)

import json
for t in texts:
    prompt_llm= """
    json: GET THE 
    MAIN IDEA!! You are to extract important entities, their definitions, overarching ideas,  and their relationships with one another from this paper. Don't copy the sentence
    word for word and split it up. Do not take all random relationships. Identify relationships that can help clinicians with diagnosis. . Will/May need to look across sections. Refrain from forming relationships unless text explicitly highlights or suggests that.
    
    For example, do not say a medicine is/isn't effective unless the text explicity says so
    
    Understand the sentence and determine the relationships between the subject and objects
    
    DO NOT use the word 'Depression' ALONE as a subject or object. Each block of text emphasizes different types of depression and you must include the type of depression the relationship is referring to.
    If the text is referring to all types of depression for that relationship, label it as "All Depression"
    The idea is that these relationships can be pieced together into a knowledge graph that a clinician can trace. Keep this in mind as you extract.
    
    Do not use any relationships other than these: 
    1. treatment:  IPT --> treatment for -> depression
    2. symptom: sadness -> symptom of -> depression
    3. example: medA --> example of -> antidepressant
    4. not effective: medA -> not effective for --> depression
    5. effective: medA -> effective for -> depression
    6. more effective: medA -> more effective than -> medB
    7. less effective: medA -> less effective than -> medB
    7. as effective as : medA -> as effective as -> medB
    8. side effects: sneezing --> side effect of -> medA
    9. treatment:  meditate --> treatment to -> reduce depression
    10. Same efficacy: medA -> Same efficacy as -> no meds
    11. Recommended first-line treatment: TreatA --> Recommended first-line treatment for --> illnesses
    12. High Efficiency: MedA -> High Efficiency for -> DiseaseA
    13. Less commonly used: MedB -> Less commonly used for -> Depression
    14. correlated: DiseaseA -> correlated with -> ProblemA
    15. More common than ... in ... : DiseaseA -> More common than -> DiseaseB -> in -> younger patients
    16. Concurrent comorbidity: Disease A -> Concurrent comorbidity -> IllnessC
    17. results: DiseaseA -> results -> faster treatment in young people
    18. predictor of: use of medA -> predictor of -> diseaseB
    19. Associateed with: Disease A -> associted with -> diases B
    20. definition: therapyA -> definition -> defA
    21: Correlation: ilnnessA -> correlation -> ilnessB
    22. Most common treatment: DiseaseA -> Most common treatment -> MedA
    23. Second most common treatment: DiseaseA - Second most common treatment -> MedA
    24. Use: MedA -> use -> decreasing symptomA or medA -> used on -> patients with problem B
    25. Properties: medC -> properties -> reduces this symptom by inihibiting that neuron
    26: form of: IllnessA -> form of -> CategroyA
    26. goal: MedA -> goal -> reduce anger
    27: requirements: treatmentA -> requirements -> patient under 23
    
    
    THESE SHOULD NOT BE THE SUBJ OR OBJ!!
    Each of these relationships is the base from which you can deviate a bit. Ex you can say treatment for, treatment with, treats, etc but stick 
    to the base idea of treating. Same applies for all relations    

    Examples: "bupropion is more likely than some ssris to lead to minimal weight gain or even weight loss."
        subj: bupropion 
        rel: side effects 
        obj: more likely than some ssris to lead to minimal weight gain or even weight loss minimal weight gain or even weight loss
    Examples: "venlafaxine's was seen to follow with less mental clarity than other medicines"
        subj: venlafaxine
        rel: side effect
        obj: less mental clarity



    Example:  As shown in Table 2, remission rates were significantly lower in patients with anxious depression,
    according to both the HAM-D criterion (22.2% versus 33.4%) and the QIDS-SR criterion (27.5% versus 38.9%). 
    Response rates were also significantly lower for patients with anxious depression (41.7% versus 52.8%). "
    
    Gives the rel(s): Anxious Depression -> finding -> low remission rates  and Anxious Depression -> finding -> low response rates 
    
    Present 1 dataframe in an easily readable and visually appealing format.
    The dataframe should explain the relationships between all subjects.
    It should have a 4 headers, subj, rel, obj, reference. Reference should hold the exact sentence(s) (in quotes) that you used to get the relationship
    If you have object2 as a header, you must have a non-empty rel2 as a header. If thats not possible place the relationship in the next row
    
    
    return a pandas dataframe structure called paper1
    Return a form of a dictionary of lists.
    Avoid entities and relationships that are more than a few words. If you need to list multiple drug names or treatments, list them in different rows
    
    
    """
    
    prompt=f"Using this information: {t}, answer the following question: {prompt_llm}"

    chat_completion=client.chat.completions.create(
        temperature = 0.2,
        messages=[
            {
                "role": "system",
                "content": prompt_llm,
                },
            {
                "role": "user",
                "content": prompt,
                }
            ],
        model= modeltype,
    )
    resp=chat_completion.choices[0].message.content
    start_idx=resp.find("data = ")
    if resp[start_idx+7] == '[':
        end_idx=resp[start_idx:].find(']')
    else:
        end_idx=resp[start_idx:].find('}')
    resp=resp[start_idx+7:start_idx+end_idx+1]
    print(eval(resp))
    
    df=pd.concat([df, pd.DataFrame(eval(resp))])
    df=df.drop_duplicates(subset=['subj', 'obj', 'rel'], keep='first')
    print("\n\n\n")






'\nimport json\nfor t in texts:\n    prompt_llm= """\n    json: GET THE \n    MAIN IDEA!! You are to extract important entities, their definitions, overarching ideas,  and their relationships with one another from this paper. Don\'t copy the sentence\n    word for word and split it up. Do not take all random relationships. Identify relationships that can help clinicians with diagnosis. . Will/May need to look across sections. Refrain from forming relationships unless text explicitly highlights or suggests that.\n    \n    For example, do not say a medicine is/isn\'t effective unless the text explicity says so\n    \n    Understand the sentence and determine the relationships between the subject and objects\n    \n    DO NOT use the word \'Depression\' ALONE as a subject or object. Each block of text emphasizes different types of depression and you must include the type of depression the relationship is referring to.\n    If the text is referring to all types of depression for that rela

In [55]:
for col in df.columns:
    df[col] = df[col].map(str.lower) #.drop_duplicates(subset=['subj', 'rel', 'obj'])
df = df.drop_duplicates(subset=['subj', 'rel', 'obj']).reset_index()

In [36]:
df =pd.read_csv("Saved_Rels.csv")
df=df.drop(columns=['index','Unnamed: 0'])
df


,subj,rel,obj,reference
0,ssris,recommended first-line treatment for,all depression,most guideline currently recommend ssris as th...
1,tca,less effective than,ssris in hospitalized patients with severe mdd...,some tcas can be more effective than ssris whe...
2,maois,less commonly used for,all depression due to side effects,"due to their side effects, maois have become l..."
3,venlafaxine,as effective as,tcas,venlafaxine’s efficacy is comparable to that o...
4,duloxetine,as effective as,ssris,"in individual studies, venlafaxine and duloxet..."
...,...,...,...,...
221,bupropion and ssris,effective for,major depressive disorder with high levels of ...,efficacy of bupropion and ssris in the treatme...
222,"vilazodone, levomilnacipran, and vortioxetine",effective for,major depressive disorder,"a review of the clinical efficacy, safety and ..."
223,trazodone,less commonly used for,depression,trazodone: a 5-year review of antidepressant e...
224,ketamine,treatment for,treatment-resistant depression,a consensus statement on the use of ketamine i...


Eval

In [ ]:
LangChainStringEvaluator("string_distance", config={"distance": "damerau_levenshtein" })

In [ ]:
!git branch main

In [ ]:
!pwd

/content


In [ ]:

!git add /content/
!git remote add origin "https://github.com/rishikasrinivas/KnowledgeGraphMentalHealth.git"
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"
!git commit -m "GPT code w/ trial kg"
!git push -u origin main

fatal: pathspec 'Brightside' did not match any files
error: remote origin already exists.
On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	KnowledgeGraphMentalHealth/

nothing added to commit but untracked files present (use "git add" to track)
fatal: could not read Username for 'https://github.com': No such device or address
